In [683]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import yfinance as yf
from datetime import datetime, timedelta

### Inputs importantes:

In [684]:
#Variáveis do modelo
buy_or_sell = 'sell' #'buy' or 'sell' setup 
data_type = 'adj_close' #'close' or 'adj_close'
start_date = '2023-02-01' #start date of backtest
start_date = datetime.strptime(start_date, '%Y-%m-%d')
start_date = start_date - pd.tseries.offsets.BDay(80)
eden_type = 'my' # 'my' == mm8 > mm80 // 'asc_mm' == mm 8 and 80 ascending // 'all' == both conditions
is_eden = 'n' #'sim' ou 'nao'
target_type = 'fixed' # 'amp' or 'fixed'
multiplier = 1.61 # if target_type = 'fixed', Multiply the risk to calculate the target. Normally 1.61 or 2

#### Inserção dos Ativos

In [685]:
stocks_list = ["CIEL3"]#,"ITSA4", "ALUP11","SUZB3", "CCRO3", "BBSE3", "JBSS3", "ENBR3", "MRVE3", "ASAI3", "CRFB3", "TOTS3", "GGBR4", "TIMS3", "IRBR3", "USIM5", "BRFS3", "BBDC4", "BRAP4", "ABEV3", "UGPA3", "PETR3", "VBBR3", "BPAC11", "ITSA4", "BPAN4", "CSAN3", "DXCO3", "TAEE11", "PCAR3", "CIEL3", "RENT3", "PRIO3", "EGIE3", "HAPV3", "LWSA3", "LREN3", "FLRY3", "CPLE6", "RRRP3", "ALPA4", "CASH3", "COGN3", "ENGI11", "VALE3", "PETZ3", "SUZB3", "EMBR3", "MRFG3", "KLBN11", "AZUL4", "CMIN3", "AMER3", "ELET6", "BRKM5", "BEEF3", "CSNA3", "MGLU3", "SLCE3", "PETR4", "POSI3", "GOLL4", "SBSP3", "CYRE3", "EQTL3", "RDOR3", "GOAU4", "WEGE3", "ECOR3", "ARZZ3", "NTCO3", "CPFE3", "IGTI11", "EZTC3", "B3SA3", "CMIG4", "RAIL3", "RAIZ4", "HYPE3", "ITUB4", "ENEV3", "BBAS3", "VIIA3", "CVCB3", "SOMA3", "VIVT3", "MULT3", "SMTO3", "YDUQ3", "SANB11"]
stocks_list = [i + '.SA' for i in stocks_list]

In [686]:
# Validar tickers descontinuados
if len(stocks_list) > 1:
    ticker_valid = yf.download(stocks_list, start = '2024-01-01', end = '2024-01-31', period = "1d")
    ticker_valid = ticker_valid['Close']
    ticker_valid.dropna(axis = 1, inplace=True)
    stocks_list = ticker_valid.columns

# Funções Auxiliares

### Definindo o algoritmo para simular as operações

In [687]:
import math

# Create a function to round any number to the smalles multiple of 100
def round_down(x):
    return int(math.floor(x / 100.0)) * 100

def backtest_algorithm_buy(
    df,
    capital_exposure,
    initial_capital):

    # List with the total capital after every operation
    total_capital = [initial_capital]

    # List with profits for every operation
    all_profits = [] 

    ongoing = False
    for i in range(0,len(df)):
        if ongoing == True:
            if (df["Open"][i] >= target) | (df["Open"][i] <= stop): 
                exit = df["Open"][i]

                profit = shares * (exit - entry)
                # Append profit to list and create a new entry with the capital
                # after the operation is complete
                all_profits += [profit]
                current_capital = total_capital[-1] # current capital is the last entry in the list
                total_capital += [current_capital + profit]

                ongoing = False

            elif df["Low"][i] <= stop: 
                exit = stop

                profit = shares * (exit - entry)
                # Append profit to list and create a new entry with the capital
                # after the operation is complete
                all_profits += [profit]
                current_capital = total_capital[-1] # current capital is the last entry in the list
                total_capital += [current_capital + profit]

                ongoing = False

            elif df["High"][i] >= target: 
                exit = target

                profit = shares * (exit - entry)
                # Append profit to list and create a new entry with the capital
                # after the operation is complete
                all_profits += [profit]
                current_capital = total_capital[-1] # current capital is the last entry in the list
                total_capital += [current_capital + profit]

                ongoing = False

        else:
 
            if ~(np.isnan(df["buy_price"][i])):
                entry = df["buy_price"][i]
                stop = df["stop"][i]
                
                if df["Low"][i] > stop: 
                    ongoing = True
                    risk = entry - stop
                    target = df["target"][i]
                    shares = round_down(capital_exposure / risk)

    return all_profits, total_capital

def backtest_algorithm_sell(
    df,
    capital_exposure,
    initial_capital):

    # List with the total capital after every operation
    total_capital = [initial_capital]

    # List with profits for every operation
    all_profits = [] 

    ongoing = False
    for i in range(0,len(df)):
        if ongoing == True:
            if (df["Open"][i] <= target) | (df["Open"][i] >= stop): 
                exit = df["Open"][i]

                profit = shares * (exit - entry)
                # Append profit to list and create a new entry with the capital
                # after the operation is complete
                all_profits += [profit]
                current_capital = total_capital[-1] # current capital is the last entry in the list
                total_capital += [current_capital + profit]

                ongoing = False
                
            elif df["High"][i] >= stop: 
                exit = stop

                profit = shares * (exit - entry)
                # Append profit to list and create a new entry with the capital
                # after the operation is complete
                all_profits += [profit]
                current_capital = total_capital[-1] # current capital is the last entry in the list
                total_capital += [current_capital + profit]

                ongoing = False

            elif df["Low"][i] <= target: 
                exit = target

                profit = shares * (exit - entry)
                # Append profit to list and create a new entry with the capital
                # after the operation is complete
                all_profits += [profit]
                current_capital = total_capital[-1] # current capital is the last entry in the list
                total_capital += [current_capital + profit]

                ongoing = False
            print(total_capital)


        else:
            print('oi')
            if ~(np.isnan(df["sell_price"][i])):
                entry = df["sell_price"][i]
                stop = df["stop"][i]
                
                if df["Low"][i] > stop: 
                    ongoing = True
                    risk = entry - stop
                    target = df["target"][i]
                    shares = round_down(capital_exposure / risk)

    return all_profits, total_capital

### Calculando a estatística e a curva de capital

In [688]:
def get_drawdown(data, column = "Close"):
    data["Max"] = data[column].cummax()
    data["Delta"] = data['Max'] - data[column]
    data["Drawdown"] = 100 * (data["Delta"] / data["Max"])
    max_drawdown = data["Drawdown"].max()
    return max_drawdown

def strategy_test(all_profits, total_capital):
    if len(all_profits) > 0:
        gains = sum(x >= 0 for x in all_profits)
        losses = sum(x < 0 for x in all_profits)
        num_operations = gains + losses
        pct_gains = 100 * (gains / num_operations)
        pct_losses = 100 - pct_gains
        total_profit = sum(all_profits)
        pct_profit = (total_profit / total_capital[0]) * 100
        
        # Compute drawdown
        total_capital = pd.DataFrame(data=total_capital, columns=["total_capital"])
        drawdown = get_drawdown(data=total_capital, column="total_capital")

        # Compute profit per operation
        profit_per_operation = pct_profit / num_operations

        # Expeceted Value
        all_positives = [x for x in all_profits if x >= 0]
        if len(all_positives) > 0:
            average_gain = sum(all_positives) / len(all_positives)
        else:
            average_gain = 0

        all_negatives = [x for x in all_profits if x < 0]
        if len(all_negatives) > 0:
            average_loss = sum(all_negatives) / len(all_negatives)
        else:
            average_loss = 0
            
        num_operations = len(all_profits)
        pct_gains = (len(all_positives) / num_operations)
        pct_losses = 1 - pct_gains
        
        expected_value = (average_gain * pct_gains) + (average_loss * pct_losses)
    else:
        total_profit = 0
        pct_profit = 0
        drawdown = 0
        num_operations = 0
        profit_per_operation = 0
        gains = 0
        pct_gains = 0
        losses = 0
        pct_losses = 0
        expected_value = 0

    return {
        "total_profit": total_profit,
        "pct_profit": pct_profit,
        "pct_drawdown": drawdown,
        "num_operations": num_operations,
        "pct_profit_per_operation": profit_per_operation,
        "num_gains": gains ,
        "pct_gains": pct_gains,
        "num_losses": losses,
        "pct_losses": pct_losses, 
        "expected_value": expected_value
    }
def capital_plot(total_capital, all_profits):
  all_profits = [0] + all_profits # make sure both lists are the same size
  cap_evolution = pd.DataFrame({'Capital': total_capital, 'Profit': all_profits})
  plt.title("Curva de Capital")
  plt.xlabel("Total Operações")
  cap_evolution['Capital'].plot()

# Ajuste do DF para chamada das Funções de modo Escalável

## 123 de Compra

In [689]:
if buy_or_sell == 'buy':
    df_stats = pd.DataFrame()
    for stock in stocks_list:
        df = yf.download(stock, start = start_date, interval = '1d')
        if data_type == 'adj_close':
            df.drop(['Close','Volume'], axis=1, inplace=True)
            df.rename({"Adj Close":"Close"},inplace=True,axis=1)
        else:
            df.drop(['Adj Close','Volume'], axis=1, inplace=True)

    ### Definindo o candle sinal
        condition_1 = df["Low"] > df["Low"].shift(1)
        condition_2 = df["Low"].shift(1) < df["Low"].shift(2)
        df["signal"] = condition_1 & condition_2

    ### Calculando o Éden dos Traders
        df["mme8"] = df["Close"].rolling(window=8).mean()
        df["mme80"] = df["Close"].rolling(window=80).mean()

        if eden_type == 'my':
            df["eden"] = (df["mme8"] > df["mme80"])
        elif eden_type == 'asc_mms':
            df["eden"] = (df["mme8"] > df["mme8"].shift(1)) & (df["mme80"] > df["mme80"].shift(1))
        else:
            df["eden"] = (df["mme8"] > df["mme8"].shift(1)) & (df["mme80"] > df["mme80"].shift(1)) & (df["mme8"] > df["mme80"])
        df.dropna(inplace=True)

    ### Definindo o preço de compra
        condition_1 = df["signal"].shift(1) == True
        condition_2 = df["eden"].shift(1) == True 
        condition_3 = df["High"] > df["High"].shift(1)
        tick = 0.01

        if is_eden == 'sim':
            df["buy_price"] = np.where(
            condition_1 & condition_2 & condition_3, 
            np.where(df["Open"] > df["High"].shift(1), df["Open"], df["High"].shift(1) + tick),
            np.nan)
        else:
            df["buy_price"] = np.where(
            condition_1 & condition_3, 
            np.where(df["Open"] > df["High"].shift(1), df["Open"], df["High"].shift(1) + tick),
            np.nan)
        
    ### Definindo o alvo
        max_high = df["High"].rolling(3).max()
        min_low = df["Low"].rolling(3).min()
        amplitude = (max_high.shift(1) - min_low.shift(1))
        fixed_risk = ((df["High"].shift(1) +tick) - (df["Low"].shift(2) - tick)) * multiplier
        entry = (df["High"].shift(1)) + tick 
        if target_type == 'fixed':
            df["target"] = fixed_risk + entry
        else:
            df["target"] =  amplitude + entry

    ### Definindo o stop
        df["stop"] = df["Low"].shift(2) - tick

    ### Realizando o Backtest
        all_profits, total_capital = backtest_algorithm_buy(
        df=df,
        capital_exposure=1000,
        initial_capital=100000)
        statistics = strategy_test(all_profits, total_capital)
        statistics = pd.DataFrame.from_dict(statistics, orient='index').round(2)
        df_stats[stock] = statistics
    df_stats = df_stats.T

## 123 de Venda

In [690]:
if buy_or_sell == 'sell':
    df_stats = pd.DataFrame()
    for stock in stocks_list:
        df = yf.download(stock, start = start_date, interval = '1d')
        if data_type == 'adj_close':
            df.drop(['Close','Volume'], axis=1, inplace=True)
            df.rename({"Adj Close":"Close"},inplace=True,axis=1)
        else:
            df.drop(['Adj Close','Volume'], axis=1, inplace=True)

    ### Definindo o candle sinal
        condition_1 = df["High"] < df["High"].shift(1)
        condition_2 = df["High"].shift(1) > df["High"].shift(2)
        df["signal"] = condition_1 & condition_2

    ### Calculando o Éden dos Traders
        df["mme8"] = df["Close"].rolling(window=8).mean()
        df["mme80"] = df["Close"].rolling(window=80).mean()

        if eden_type == 'my':
            df["eden"] = (df["mme8"] < df["mme80"])
        elif eden_type == 'asc_mms':
            df["eden"] = (df["mme8"] < df["mme8"].shift(1)) & (df["mme80"] < df["mme80"].shift(1))
        else:
            df["eden"] = (df["mme8"] < df["mme8"].shift(1)) & (df["mme80"] < df["mme80"].shift(1)) & (df["mme8"] < df["mme80"])
        df.dropna(inplace=True)

    ### Definindo o preço de VENDA
        condition_1 = df["signal"].shift(1) == True
        condition_2 = df["eden"].shift(1) == True 
        condition_3 = df["Low"] < df["Low"].shift(1)
        tick = 0.01

        if is_eden == 'sim':
            df["sell_price"] = np.where(
            condition_1 & condition_2 & condition_3, 
            np.where(df["Open"] < df["Low"].shift(1), df["Open"], df["Low"].shift(1) - tick),
            np.nan)
        else:
            df["sell_price"] = np.where(
            condition_1 & condition_3, 
            np.where(df["Open"] < df["Low"].shift(1), df["Open"], df["Low"].shift(1) - tick),
            np.nan)
        
    ### Definindo o alvo
        max_high = df["High"].rolling(3).max()
        min_low = df["Low"].rolling(3).min()
        amplitude = (max_high.shift(1) - min_low.shift(1))
        fixed_risk = ((df["High"].shift(2) +tick) - (df["Low"].shift(1) - tick)) * multiplier
        entry = (df["Low"].shift(1)) - tick 
        if target_type == 'fixed':
            df["target"] = entry - fixed_risk
        else:
            df["target"] =  entry - amplitude
    ### Definindo o stop
        df["stop"] = df["High"].shift(2) + tick

    ### Realizando o Backtest
        all_profits, total_capital = backtest_algorithm_sell(
        df=df,
        capital_exposure=1000,
        initial_capital=100000)
        statistics = strategy_test(all_profits, total_capital)
        statistics = pd.DataFrame.from_dict(statistics, orient='index').round(2)
        df_stats[stock] = statistics
    df_stats = df_stats.T

[*********************100%***********************]  1 of 1 completed
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi
oi


In [694]:
df.sort_values('sell_price')

,Open,High,Low,Close,signal,mme8,mme80,eden,sell_price,target,stop
Date,,,,,,,,,,,
2023-10-30,3.45,3.50,3.40,3.327287,False,3.438519,3.748641,True,3.42,3.114100,3.61
2023-10-03,3.54,3.54,3.37,3.259580,False,3.300688,3.957056,True,3.46,3.282900,3.57
2023-10-18,3.51,3.55,3.46,3.404665,False,3.451818,3.835678,True,3.50,2.872100,3.89
2023-10-23,3.60,3.71,3.56,3.540078,False,3.518315,3.805065,True,3.57,3.167500,3.82
2023-09-15,3.62,3.65,3.53,3.453027,False,3.549751,4.136963,True,3.60,3.310200,3.78
...,...,...,...,...,...,...,...,...,...,...,...
2024-03-06,5.40,5.44,5.34,5.380000,False,5.382500,4.620748,False,NaN,5.201200,5.41
2024-03-07,5.35,5.40,5.35,5.350000,True,5.372500,4.640920,False,NaN,5.185101,5.42
2024-03-11,5.34,5.37,5.33,5.350000,False,5.365000,4.681514,False,NaN,5.122900,5.41


In [692]:
df_stats.sort_values('expected_value').tail(20)

,total_profit,pct_profit,pct_drawdown,num_operations,pct_profit_per_operation,num_gains,pct_gains,num_losses,pct_losses,expected_value
CIEL3.SA,0,0,0,0,0,0,0,0,0,0


# Mlhorias a serem feitas

- Adicionar filtro Ibov
- Alterar modelo de backtest
- Adicionar filtro Gap
- Adicionar filtro amplitude (se a amp entre o preço de entrada e o stop forem muito longos)
- Adicionar stop por tempo